# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/city_dataframe.csv"
DF = pd.read_csv(file_path)
DF.head()

,Name,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Date
0,Cape Town,-33.9258,18.4232,59.918,72,0,5.14,1658948018
1,Pevek,69.7008,170.3133,34.466,91,100,1.82,1658948305
2,Tuatapere,-46.1333,167.6833,43.934,95,100,0.73,1658948305
3,Chifeng,42.2683,118.9636,67.874,85,100,0.66,1658948306
4,Srednekolymsk,67.4500,153.6833,40.730,73,83,3.85,1658948306


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configures gmaps
gmaps.configure(api_key=g_key)

In [4]:
# heatmap of humidity
locations = DF[["Latitude", "Longitude"]]
humidity = DF["Humidity"]
figure = gmaps.figure()

# add heatmap layer to figure
heatmapLayer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
figure.add_layer(heatmapLayer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealDF = DF.loc[(DF["Maximum Temperature"] > 70) & (DF["Maximum Temperature"] < 80) & (DF["Wind Speed"] < 10) & (DF["Cloudiness"] == 0)].dropna()
idealDF.head()

,Name,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed,Date
51,Ponta do Sol,32.6667,-17.1000,76.712,75,0,2.79,1658947724
58,Ketchikan,55.3422,-131.6461,70.268,64,0,4.63,1658947725
124,Atuona,-9.8000,-139.0333,77.252,67,0,9.34,1658947734
132,Mahon,39.8885,4.2658,79.358,53,0,2.06,1658948276
180,Palestine,31.9216,35.2033,74.354,67,0,2.02,1658948139


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotelDF = idealDF[["Name", "Latitude", "Longitude"]]
hotelDF["Hotel Name"] = ""
hotelDF.head()


/var/folders/c1/_06r37k97pz1w7fp3ym2jfvw0000gn/T/ipykernel_2538/572100223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDF["Hotel Name"] = ""


,Name,Latitude,Longitude,Hotel Name
51,Ponta do Sol,32.6667,-17.1000,
58,Ketchikan,55.3422,-131.6461,
124,Atuona,-9.8000,-139.0333,
132,Mahon,39.8885,4.2658,
180,Palestine,31.9216,35.2033,


In [8]:
# set params to search for hotel
params = {
    "radius" : 5000,
    "types" : "lodging",
    "key" : g_key
}

# loop through hotelDF using iterrows()
for index, row in hotelDF.iterrows():
    # get lat and lng from each row
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # use the lat and lng to set the location value in params dict
    params["location"] = f"{lat},{lng}"
    
    # use google apis places base usrl to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    #pprint(name_address) # print ["results"][0]["name"]
    
    try:
        # populate the hotel name
        hotelDF.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Could not find close-by lodging in {row['City']}.")
        pass
hotelDF

,Name,Latitude,Longitude,Hotel Name
51,Ponta do Sol,32.6667,-17.1000,Hotel do Campo
58,Ketchikan,55.3422,-131.6461,Snow's Cove Lodge
124,Atuona,-9.8000,-139.0333,Villa Enata
132,Mahon,39.8885,4.2658,ARTIEM Capri
180,Palestine,31.9216,35.2033,Grand Park Hotel
216,Fundão,-19.9341,-40.4047,City Hotel
241,Ürümqi,43.8010,87.6005,Hua Ling Grand Hotel Urumqi
255,Novopokrovka,49.8425,36.5477,Пудж дом
287,Los Llanos de Aridane,28.6585,-17.9182,Valle Aridane
341,Peniche,39.3558,-9.3811,Hotel Star inn Peniche


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotelDF.iterrows()]
locations = hotelDF[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markerLayer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(markerLayer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))